Import Sentinel-1 Data from Google Earth Engine

This will download S1 data directly into your Google Drive.

Follow these steps for installing the EE library and troubleshooting authentication issues:
https://developers.google.com/earth-engine/guides/python_install

In [1]:
# import libraries

import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time
from speckle_filter import leesigma

In [2]:
# get authentication credentials to initialize ee

ee.Authenticate()
ee.Initialize()

Enter verification code: 

Successfully saved authorization token.


In [3]:
# define geometry and dates for data collection

# region of interest
roi = ee.Geometry.Rectangle([70.7980547394194417, 23.2880385161501735, 71.5060515087354958, 23.9024834537972986])  # Nanda Bet, India

# dates of interest
d1 = '2020-01-01'
d2 = '2020-02-01'

In [4]:
# import s1 data

s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                 .filter(ee.Filter.eq('instrumentMode', 'IW'))
                 .filterMetadata('resolution_meters', 'equals', 10)
                 .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                 .filter(ee.Filter.date(d1,d2))
                 .filterBounds(roi))
print('Number of images in s1 collection:', s1_collection.size().getInfo())

Number of images in s1 collection: 3


In [5]:
# get dates of s1 images

acq_times = s1_collection.aggregate_array('system:time_start').getInfo()
date_list = [time.strftime('%m-%d-%Y', time.gmtime(t/1000)) for t in acq_times]
print(date_list)

['01-03-2020', '01-15-2020', '01-27-2020']


In [6]:
# implement speckle filter and get vv/vh ratio

def speckle(img):
    from speckle_filter import leesigma
    k = 5
    vv = img.select('VV')
    vh = img.select('VH')
    vv_vh = vv.divide(vh).rename('VV_VH')
    vv_filtered = leesigma(vv,k).rename('VV_filtered')
    vh_filtered = leesigma(vh,k).rename('VH_filtered')
    vv_vh_filtered = vv_filtered.divide(vh_filtered).rename('VV_VH_filtered')
    img = img.addBands(ee.Image([vv_vh, vv_filtered, vh_filtered, vv_vh_filtered]))
    return img

s1_collection = s1_collection.map(speckle)

In [7]:
# print band names

print(s1_collection.first().bandNames().getInfo())

['VV', 'VH', 'angle', 'VV_VH', 'VV_filtered', 'VH_filtered', 'VV_VH_filtered']


In [23]:
# export all n images in image collection

n = s1_collection.size().getInfo()  #number of images in image collection
s1_collection_list = s1_collection.toList(n)
scale = 10        #resolution of exported image in meters
maxPixels = 2e10  #needs to be high for high resolution or export will fail
task_id = []      #for tracking tasks

for i in range(0,n):
    img = ee.Image(s1_collection_list.get(i)).select(['VV_filtered','VH_filtered','VV_VH_filtered']).clip(roi)
    txt = 's1_' + date_list[i] + '_' + str(scale) + 'm'
    print(txt, date_list[i])
    task = ee.batch.Export.image.toDrive(image=img,
                                        description=txt,
                                        scale=scale,
                                        region=roi,
                                        fileNamePrefix=txt,
                                        crs='EPSG:4326',
                                        fileFormat='GeoTIFF',
                                        maxPixels=maxPixels)
    task.start()
    task_id.append(task.id)
    

s1_01-03-2020_10m 01-03-2020
s1_01-15-2020_10m 01-15-2020
s1_01-27-2020_10m 01-27-2020


In [ ]:
# list status of recent tasks

task.list()